In [1]:
import numpy as np
import matplotlib.pyplot as plt

from synthlearners import Synth
from synthlearners.simulator import (
    SimulationConfig,
    PanelSimulator,
    FactorDGP,
)
from synthlearners.plotting import SynthPlotter

## Basic use with Panel Matrices

In [2]:
config = SimulationConfig(
    N=100,
    T=50,
    T_pre=40,
    n_treated=10,
    selection_mean=1.0,
    treatment_effect=0.5,
    dgp=FactorDGP(K=2, sigma=1, time_fac_lb=-0.1, time_fac_ub=0.1, trend_sigma=0.0001),
)
simulator = PanelSimulator(config)
Y, Y_0, L, treated_units = simulator.simulate()

In [3]:
treated_units

array([89, 32, 13, 80, 85, 28, 82, 45,  9, 64])

In [4]:
from synthlearners.mcnnm import MatrixCompletionEstimator

In [5]:
# Create a mask indicating which entries are observed.
mask = np.ones_like(Y)
missing_periods = config.T-config.T_pre

# For treated units, suppose the outcomes in the last N periods are missing.
mask[treated_units, -missing_periods:] = 0  

In [6]:
mask[treated_units[0]]

array([1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [7]:
estimator = MatrixCompletionEstimator(
    lambda_param=1e-3,
    max_iter=500,
    tol=1e-6,
    verbose=True
)
estimator.fit(Y, mask)
Y_completed = estimator.completed_matrix_

Iteration   1, relative change = 5562318550.187082
Iteration   2, relative change = 0.011674
Iteration   3, relative change = 0.008889
Iteration   4, relative change = 0.006807
Iteration   5, relative change = 0.005247
Iteration   6, relative change = 0.004069
Iteration   7, relative change = 0.003174
Iteration   8, relative change = 0.002487
Iteration   9, relative change = 0.001957
Iteration  10, relative change = 0.001546
Iteration  11, relative change = 0.001225
Iteration  12, relative change = 0.000973
Iteration  13, relative change = 0.000775
Iteration  14, relative change = 0.000619
Iteration  15, relative change = 0.000495
Iteration  16, relative change = 0.000397
Iteration  17, relative change = 0.000319
Iteration  18, relative change = 0.000257
Iteration  19, relative change = 0.000207
Iteration  20, relative change = 0.000167
Iteration  21, relative change = 0.000135
Iteration  22, relative change = 0.000109
Iteration  23, relative change = 0.000089
Iteration  24, relative c

In [8]:
# Counterfactual Values for imputed periods for single treated example
Y_0[treated_units[0], -missing_periods:]

array([-0.76488391, -0.15174888, -0.13119737,  0.30966483,  0.12721515,
        0.81209996,  0.97510635, -1.01615275,  0.85462799, -1.3490142 ])

In [9]:
# Observed Value for imputed periods from single treated example
Y[treated_units[0], -missing_periods:]

array([-0.26488391,  0.34825112,  0.36880263,  0.80966483,  0.62721515,
        1.31209996,  1.47510635, -0.51615275,  1.35462799, -0.8490142 ])

In [10]:
# MC Values for imputed periods for single treated example
Y_completed[treated_units[0], -missing_periods:]

array([ 0.10977991, -0.30386904, -0.35959057,  0.36566895,  0.2987112 ,
        0.24407495,  0.04952359, -0.35429608,  0.3084663 , -0.79258891])

In [11]:
baseline_bias = Y_completed[treated_units[0], -missing_periods:]-Y_0[treated_units[0], -missing_periods:]
print('Baseline Bias of MC value relative to actual counterfactual',np.mean(baseline_bias))

Baseline Bias of MC value relative to actual counterfactual -0.009983685543181752


In [12]:
te = Y[treated_units[0], -missing_periods:]-Y_completed[treated_units[0], -missing_periods:]
print('Estimated Treatment Effect of Observed relative to MC Value',np.mean(te))

Estimated Treatment Effect of Observed relative to MC Value 0.5099836855431817


In [13]:
print('Treatment Effect Bias',np.abs(np.mean(te)-config.treatment_effect))

Treatment Effect Bias 0.009983685543181742
